In [13]:
import math
from abc import ABC, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from mido import MidiFile
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, fftconvolve
class Oscillator(ABC):

    def __init__(self, freq=440, phase=0, amp=1, sample_rate=44100, wave_range=(-1, 1), detune=0):
        self._freq = freq
        self._amp = amp
        self._phase = phase
        self._sample_rate = sample_rate
        self._wave_range = wave_range
        self._detune = detune

        # Properties that will be changed
        self._f = freq + detune
        self._a = amp
        self._p = phase

    @property
    def init_freq(self):
        return self._freq

    @property
    def init_amp(self):
        return self._amp

    @property
    def init_phase(self):
        return self._phase

    @property
    def freq(self):
        return self._f

    @freq.setter
    def freq(self, value):
        self._f = value + self._detune
        self._post_freq_set()

    @property
    def amp(self):
        return self._a

    @amp.setter
    def amp(self, value):
        self._a = value
        self._post_amp_set()

    @property
    def phase(self):
        return self._p

    @phase.setter
    def phase(self, value):
        self._p = value
        self._post_phase_set()

    def _post_freq_set(self):
        pass

    def _post_amp_set(self):
        pass

    def _post_phase_set(self):
        pass

    @abstractmethod
    def _initialize_osc(self):
        pass

    @staticmethod
    def squish_val(val, min_val=0, max_val=1):
        return (((val + 1) / 2 ) * (max_val - min_val)) + min_val

    @abstractmethod
    def __next__(self):
        return None

    def __iter__(self):
        self.freq = self._freq
        self.phase = self._phase
        self.amp = self._amp
        self._initialize_osc()
        return self

class SineOscillator(Oscillator):

    def _initialize_osc(self):
        self._i = 0
        self._step = (2 * math.pi * self._f) / self._sample_rate

    def __next__(self):
        val = math.sin(self._i + self._p)
        self._i = (self._i + self._step) % (2 * math.pi)
        return val * self._a

class SquareOscillator(Oscillator):

    def __init__(self, freq=440, phase=0, amp=1, sample_rate=44100, wave_range=(-1, 1), threshold=0, detune=0):
        super().__init__(freq, phase, amp, sample_rate, wave_range, detune)
        self.threshold = threshold

    def _initialize_osc(self):
        self._i = 0
        self._step = (2 * math.pi * self._f) / self._sample_rate

    def __next__(self):
        val = math.sin(self._i + self._p)
        self._i = (self._i + self._step) % (2 * math.pi)
        if val < self.threshold:
            val = self._wave_range[0]
        else:
            val = self._wave_range[1]
        return val * self._a

class SawtoothOscillator(Oscillator):

    def _post_freq_set(self):
        self._period = self._sample_rate / self._f
        self._post_phase_set()

    def _post_phase_set(self):
        self._p = ((self._p + 90) / 360) * self._period

    def _initialize_osc(self):
        self._i = 0

    def __next__(self):
        div = (self._i + self._p) / self._period
        val = 2 * (div - math.floor(0.5 + div))
        self._i = (self._i + 1) % self._period
        if self._wave_range != (-1, 1):
            val = self.squish_val(val, *self._wave_range)
        return val * self._a

class TriangleOscillator(SawtoothOscillator):

    def __next__(self):
        div = (self._i + self._p) / self._period
        val = 2 * (div - math.floor(0.5 + div))
        val = (abs(val) - 0.5) * 2
        self._i = (self._i + 1) % self._period
        if self._wave_range != (-1, 1):
            val = self.squish_val(val, *self._wave_range)
        return val * self._a

class NoiseOscillator:

    def __init__(self, amp=1, sample_rate=44100):
        self._a = amp
        self._sample_rate = sample_rate

    def __iter__(self):
        return self

    def __next__(self):
        return np.random.normal(0, 1) * self._a

class WaveAdder:

    def __init__(self, *oscillators):
        self.oscillators = oscillators
        self.n = len(oscillators)

    def __iter__(self):
        for osc in self.oscillators:
            iter(osc)
        return self

    def __next__(self):
        return sum(next(osc) for osc in self.oscillators) / self.n

def read_midi_file(file_path):
    mid = MidiFile(file_path)
    for i, track in enumerate(mid.tracks):
        print(f'Track {i}: {track.name}')
        for msg in track:
            print(msg)
    return mid

def note_to_freq(note):
    """Convert MIDI note to frequency."""
    A4 = 440.0
    return A4 * 2**((note - 69) / 12.0)

def midi_to_audio(midi_file_path, audio_file_path, oscillator_class, sample_rate=44100):
    mid = read_midi_file(midi_file_path)
    
    audio_data = np.array([])

    tempo = 500000  # default tempo in microseconds per beat
    ticks_per_beat = mid.ticks_per_beat
    seconds_per_tick = tempo / 1_000_000.0 / ticks_per_beat

    for track in mid.tracks:
        current_time = 0.0
        note_start_times = {}
        for msg in track:
            if msg.type == 'set_tempo':
                tempo = msg.tempo
                seconds_per_tick = tempo / 1_000_000.0 / ticks_per_beat
            current_time += msg.time * seconds_per_tick
            if msg.type == 'note_on' and msg.velocity > 0:
                note_start_times[msg.note] = current_time
            elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)) and msg.note in note_start_times:
                start_time = note_start_times[msg.note]
                duration_seconds = current_time - start_time
                frequency = note_to_freq(msg.note)
                oscillator = oscillator_class(freq=frequency, amp=1, sample_rate=sample_rate)
                oscillator.__iter__()  # Ensure the oscillator is properly initialized
                wave = np.array([next(oscillator) for _ in range(int(duration_seconds * sample_rate))])
                
                start_sample = int(start_time * sample_rate)
                end_sample = start_sample + len(wave)
                
                if start_sample > len(audio_data):
                    silence = np.zeros(start_sample - len(audio_data))
                    audio_data = np.concatenate((audio_data, silence))
                
                if end_sample > len(audio_data):
                    audio_data = np.concatenate((audio_data, np.zeros(end_sample - len(audio_data))))
                
                audio_data[start_sample:end_sample] += wave
                
                del note_start_times[msg.note]

    if audio_data.size == 0:
        print("No audio data was generated.")
        return

    # Normalize to 16-bit range
    audio_data = np.int16(audio_data / np.max(np.abs(audio_data)) * 32767)
    write(audio_file_path, sample_rate, audio_data)
    print(f"Audio file saved at: {audio_file_path}")
def generate_ahdsr_envelope(attack_time, hold_time, decay_time, sustain_level, release_time, duration, sample_rate):
    total_samples = int(duration * sample_rate)
    attack_samples = int(attack_time * sample_rate)
    hold_samples = int(hold_time * sample_rate)
    decay_samples = int(decay_time * sample_rate)
    release_samples = int(release_time * sample_rate)
    sustain_samples = total_samples - (attack_samples + hold_samples + decay_samples + release_samples)

    if sustain_samples < 0:
        raise ValueError("AHDSR parameters are too long for the given duration")

    envelope = np.zeros(total_samples)
    # Attack
    envelope[:attack_samples] = exponential_interpolation(1e-5, 1, attack_samples)
    # Hold
    envelope[attack_samples:attack_samples + hold_samples] = 1
    # Decay
    envelope[attack_samples + hold_samples:attack_samples + hold_samples + decay_samples] = exponential_interpolation(1, sustain_level, decay_samples)
    # Sustain
    envelope[attack_samples + hold_samples + decay_samples:attack_samples + hold_samples + decay_samples + sustain_samples] = sustain_level
    # Release
    envelope[-release_samples:] = exponential_interpolation(sustain_level, 1e-5, release_samples)

    return envelope

def exponential_interpolation(start, end, num_points):
    return start * np.exp(np.log(end/start) * np.linspace(0, 1, num_points))

def apply_envelope(waveform, envelope):
    return waveform * envelope    


class EffectRack:
    def __init__(self):
        self.effects = []
    
    def add_effect(self, effect_func, *args, wet_dry=1):
        if wet_dry not in [0, 1]:
            raise ValueError("wet_dry must be either 0 or 1")
        self.effects.append((effect_func, args, wet_dry))
    
    def hard_clipping(self, x, T):
        return np.clip(x, -T, T)
    
    def compressor(self, input_signal, threshold, ratio, attack, release, sample_rate, N):
        threshold_linear = 10 ** (threshold / 20.0)
        attack_coeff = np.exp(-1.0 / (attack * sample_rate))
        release_coeff = np.exp(-1.0 / (release * sample_rate))
        output_signal = np.zeros_like(input_signal)
        avg_abs_input = np.zeros_like(input_signal)
        gain = np.ones_like(input_signal)
        
        for i in range(1, len(input_signal)):
            avg_abs_input[i] = np.mean(np.abs(input_signal[max(0, i-N+1):i+1]))
            if avg_abs_input[i] > threshold_linear:
                target_gain = threshold_linear + (avg_abs_input[i] - threshold_linear) / ratio
                if target_gain < gain[i - 1]:
                    gain[i] = attack_coeff * (gain[i - 1] - target_gain) + target_gain
                else:
                    gain[i] = release_coeff * (gain[i - 1] - target_gain) + target_gain
            else:
                gain[i] = 1.0
            output_signal[i] = gain[i] * input_signal[i]
        
        return output_signal
    
    @staticmethod
    def delay_system(input_signal, base_delay_samples, lfo_signal=None, lfo_depth=0):
        output_signal = np.zeros_like(input_signal)
        max_delay = base_delay_samples + int(lfo_depth * np.max(np.abs(lfo_signal))) if lfo_signal is not None else base_delay_samples
        buffer = np.zeros(len(input_signal) + max_delay)
        buffer[:len(input_signal)] = input_signal
        
        for i in range(len(input_signal)):
            delay = base_delay_samples
            if lfo_signal is not None:
                delay += int(lfo_depth * lfo_signal[i])
            if delay < 0:
                delay = 0
            output_signal[i] = input_signal[i] + buffer[i + delay]
            buffer[i + delay] = output_signal[i]

        return output_signal
    
    @staticmethod
    def phaser(audio, sample_rate, *args):
        # Unpack args
        rate = args[0] if len(args) > 0 else 0.5
        depth = args[1] if len(args) > 1 else 1
        feedback = args[2] if len(args) > 2 else 0.5
        n_stages = args[3] if len(args) > 3 else 4

        lfo = depth * np.sin(2 * np.pi * rate * np.arange(len(audio)) / sample_rate) 
        # Precalculate all possible delays based on the LFO
        max_delay = int(sample_rate / 4)
        all_delays = np.round((0.5 + 0.5 * lfo) * max_delay).astype(int)

        allpass_g = feedback

        for stage in range(n_stages):  
            # Create coefficients for the allpass filter
            a0 = (1 - allpass_g) / (1 + allpass_g)
            b = [a0, 1]
            a = [1, a0]

            # Apply allpass filter with the corresponding delay for each sample
            filtered_audio = np.zeros_like(audio)  # Initialize the filtered audio array
            for i, delay in enumerate(all_delays):
                delayed_sample = audio[max(0, i - delay)]
                filtered_audio[i] = lfilter(b, a, [delayed_sample])[0]

            # Mix the filtered signal with the original signal
            audio += filtered_audio

        return audio / n_stages
    
    @staticmethod
    def flanger(signal, sample_rate, delay, depth, rate, feedback=0):
       
        N = len(signal)
        delay_samples = int(sample_rate * delay)
        depth_samples = int(sample_rate * depth)
        lfo = depth_samples * (1 + np.sin(2 * np.pi * rate * np.arange(N) / sample_rate)) / 2
        flanged_signal = np.zeros(N)
        feedback_buffer = np.zeros(N)

        for n in range(N):
            delay_n = int(lfo[n])
            if n - delay_samples - delay_n < 0:
                flanged_signal[n] = signal[n]
            else:
                flanged_signal[n] = signal[n] + feedback * feedback_buffer[n - delay_samples - delay_n]
                feedback_buffer[n] = flanged_signal[n]

        return flanged_signal    
    
    @staticmethod
    def chorus(signal, sample_rate, delay, depth, rate, num_voices=3, spread=0.002):
       
        N = len(signal)
        delay_samples = int(sample_rate * delay)
        depth_samples = int(sample_rate * depth)
        lfo = np.zeros((num_voices, N))

        for i in range(num_voices):
            lfo[i] = depth_samples * (1 + np.sin(2 * np.pi * (rate + i * spread) * np.arange(N) / sample_rate)) / 2

        chorus_signal = np.zeros(N)
        for i in range(N):
            for j in range(num_voices):
                delay_n = int(lfo[j, i])
                if i - delay_samples - delay_n >= 0:
                    chorus_signal[i] += signal[i - delay_samples - delay_n]
            chorus_signal[i] /= num_voices

        return chorus_signal
    
    @staticmethod
    def digital_reverb(audio, sample_rate, delays, gains):
       
        assert len(delays) == len(gains), "Number of delays must match number of gains"

        # Ensure all pairs of delays have gcd equal to 1
        def gcd(a, b):
            while b:
                a, b = b, a % b
            return a
        
        for i in range(len(delays)):
            for j in range(i + 1, len(delays)):
                assert gcd(delays[i], delays[j]) == 1, f"Delays {delays[i]} and {delays[j]} do not have gcd 1"
        
        reverb_signal = np.zeros_like(audio)
        
        for delay, gain in zip(delays, gains):
            delayed_signal = np.zeros_like(audio)
            for i in range(delay, len(audio)):
                delayed_signal[i] = gain * audio[i - delay]
            reverb_signal += delayed_signal

        return reverb_signal
    
    @staticmethod
    def apply_convolution_reverb(input_signal, impulse_response):
  
        reverb_audio = fftconvolve(input_signal, impulse_response, mode='full')
        return reverb_audio
    
    def apply_effects(self, input_signal):
        output_signal = input_signal
        for effect_func, args, wet_dry in self.effects:
            if wet_dry == 1:
                if effect_func == EffectRack.digital_reverb:
                    delays, gains = args
                    output_signal = effect_func(output_signal, sample_rate, delays, gains)
                elif effect_func == self.apply_convolution_reverb:
                    impulse_response, = args
                    output_signal = effect_func(output_signal, impulse_response)
                else:
                    output_signal = effect_func(output_signal, *args)
        return output_signal
    
    @staticmethod
    def generate_lfo(sample_rate, duration, frequency, depth):
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        lfo_signal = np.sin(2 * np.pi * frequency * t) * depth
        return lfo_signal




In [12]:
sample_rate = 44100
duration = 5.0  # seconds
frequency = 440.0  # Hz

# Create an instance of SineOscillator
oscillator = SineOscillator(freq=frequency, amp=0.5, sample_rate=sample_rate)
oscillator.__iter__()  # Initialize the oscillator properly

# Generate the signal
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
audio_signal = np.array([next(oscillator) for _ in range(len(t))])

# Create an EffectRack instance
effect_rack = EffectRack()

# Add effects to the EffectRack
effect_rack.add_effect(EffectRack.flanger, audio_signal, sample_rate, 0.005, 0.002, 0.1, 0.5)
effect_rack.add_effect(EffectRack.compressor, audio_signal, -20, 3, 0.01, 0.1, sample_rate, 50)

# Apply effects to the audio signal
processed_signal = effect_rack.apply_effects(audio_signal)

# Plot the original and processed signals
plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.plot(t, audio_signal, label='Original Signal', color='b')
plt.title('Original Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(t, processed_signal, label='Processed Signal', color='r')
plt.title('Processed Signal with Effects')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

plt.tight_layout()
plt.show()

TypeError: EffectRack.apply_effects() missing 1 required positional argument: 'input_signal'